In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.python.compiler.mlcompute import mlcompute
# disable_eager_execution()
mlcompute.set_mlc_device(device_name='any') 
print(tf.__version__)

2.4.0-rc0


In [2]:
#import preprocessed per user dataset from onedrive prickled file
import pickle
import os
import numpy as np

path = r"/Users/nathantc5/OneDrive - The University of Sydney (Students)/CS48-CAPSTONE Project 2021 Sem1/dataset"
#create output dictionary
#enter number of user here
N_user = 1000

data = {}
for u in os.listdir(path+'/user_preprocessed_pickle')[:N_user]: #import N users

    uid = int(u[:-7])
    file_name = '/'.join([path,'/user_preprocessed_pickle', u])

    try:
        with open(file_name, 'rb') as f:
            dic = pickle.load(f)
            data[uid] = dic[uid]
    except:
        print(dic)

In [3]:
#create 2 function to extract the feature and label from array
#just to keep every clean as possible

def extract_feature(lst):
    #extract first 2 columns: session and mode location 
    return [item[:2] for item in lst]

def extract_label(lst):
    #extract the remaning columns
    return [item[2:] for item in lst]



# function to generate input for baseline model
# return(no.of users, rows of a users, feature dimension)
def generate_baseline_input(data, mode):
    data_neural = data
    data_feature = []
    data_label = []

    if mode == 'train':
        day_id = [20, 21, 22, 25] #the day for training

    elif mode == 'test':
        day_id = [26] # the day for testing

    # if candidate is None:
    candidate = data_neural.keys() #filter, and get user id

    #iterate all the users
    for u in candidate:
        #seperate feature and label list
        user_X_train = []
        user_y_label = []
        #get user's record
        sessions = data_neural[u]
        #seperate days for training and testing
        for i in day_id:
            #call specific day
            session = data_neural[u][i]
            #extract the part we want
            timeloc = extract_feature(session)[1:]
            prev_app_count = extract_label(session)[:-1]
            target = extract_label(session)[1:]
            #concate feature
            user_feature = np.concatenate((timeloc,prev_app_count),axis = 1)
            #append feature and label to corresponding list
            user_X_train.append(user_feature)
            user_y_label.append(target)
        #reason for reshape here: group 1 user at 1 dimension when append to all_user list
        user_X_train = np.array(user_X_train).reshape(-1,2002)
        user_y_label = np.array(user_y_label).reshape(-1,2000)
        #append user_data to all_use list
        data_feature.append(user_X_train)
        data_label.append(user_y_label)
    # no reshape here because when calling the variables it is easy to check how many user have been imported\
    # therefore it returns 3 dimenionsal data : (no.of users, rows of a users, feature dimension)
    data_feature = np.array(data_feature)
    data_label = np.array(data_label)

    return data_feature, data_label


In [4]:
X_train, y_train = generate_baseline_input(data, 'train')
X_test, y_test = generate_baseline_input(data,'test')
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print("X_train has {} user, each user has {} rows, each row has {} columns". format(X_train.shape[0], X_train.shape[1], X_train.shape[2]))

(870, 188, 2002)
(870, 188, 2000)
(870, 47, 2002)
(870, 47, 2000)
X_train has 870 user, each user has 188 rows, each row has 2002 columns


In [5]:
X_train = X_train.reshape(-1,2002)
y_train = y_train.reshape(-1,2000)
print(X_train.shape)
print(y_train.shape)
X_test = X_test.reshape(-1,2002)
y_test = y_test.reshape(-1,2000)
print(X_test.shape)
print(y_test.shape)

(163560, 2002)
(163560, 2000)
(40890, 2002)
(40890, 2000)


In [6]:
y_train = np.where(y_train > 0, 1, 0)
y_test = np.where(y_test > 0, 1, 0)

# MLP dataset and Model

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [8]:
BATCH_SIZE = 512
SHUFFLE_BUFFER_SIZE = 256

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [10]:
learning_rate = 0.005
#simple 3 layers model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(2002, activation="relu"),
    tf.keras.layers.Dense(1024, activation="relu"),
    tf.keras.layers.Dense(2000, activation="sigmoid")
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['AUC', 'TopKCategoricalAccuracy'])

In [11]:
model.fit(train_dataset, epochs=3)

Epoch 1/3
320/320 [==============================] - 387s 1s/step - loss: 0.2607 - auc: 0.6676 - top_k_categorical_accuracy: 0.6444
Epoch 2/3
175/320 [===============>..............] - ETA: 3:19 - loss: 0.0058 - auc: 0.7759 - top_k_categorical_accuracy: 0.3521ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/Users/nathantc5/miniforge3/envs/tensorflow_m1/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-8759dee8417e>", line 1, in <module>
    model.fit(train_dataset, epochs=3)
  File "/Users/nathantc5/miniforge3/envs/tensorflow_m1/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1100, in fit
    tmp_logs = self.train_function(iterator)
  File "/Users/nathantc5/miniforge3/envs/tensorflow_m1/lib/python3.8/site-packages/tensorflow/python/eager/def_f

TypeError: object of type 'NoneType' has no len()

In [ ]:
y_pred = model.predict(X_test)
print(y_pred.shape)

In [ ]:
def cal_ap( y_actual, y_pred, k ):
    topK = min( len(y_pred), k ) # set top k
    l_zip = list(zip(y_actual,y_pred))
    # sort y_pred by the probability of the model
    s_zip = sorted( l_zip, key=lambda x: x[1], reverse=True )
    # topk of sorted result
    s_zip_topk = s_zip[:topK] # Shape (5,2)
    # Calculation of precision
    num = 0
    rank = 0
    sumP = 0.0
    for item in s_zip_topk:
        rank += 1
        if item[0] == 1:
            num += 1
            sumP += (num*1.0)/(rank*1.0)
    ap = 0.0
    if num > 0:
        ap = sumP/(num*1.0)
    return ap   # average precision
# Take topk prediction and the ground truth


def r_k(y_actual, y_pred, k, threshold):
    topK = min( len(y_pred), k ) # set top k
    l_zip = list(zip(y_actual,y_pred))
    # sort y_pred by the probability of the model
    s_zip = sorted( l_zip, key=lambda x: x[1], reverse=True )
    # topk of sorted result
    s_zip_topk = s_zip[:topK] # Shape (5,2)
    # print(s_zip_topk)
    actual, pred = zip(*s_zip_topk)
    actual = np.where(np.array(actual) > threshold, 1, 0)
    pred = np.array(pred)
    pred_o = np.where(pred > threshold, 1, 0)
    return actual, pred_o, pred

# acc[3] += skmetrics.recall_score(actual, pred_o, average='macro') # Recall@5

In [ ]:
from sklearn import metrics as skmetrics

avg_auc = 0
avg_map = 0
avg_recall = 0
count = 0

for i in range(y_test.shape[0]):
    if (np.sum(y_test[i])> 0):
        fpr, tpr, thresholds = skmetrics.roc_curve(y_test[i], y_pred[i], pos_label=1)
        avg_auc += skmetrics.auc(fpr, tpr)
        actual, pred_o, pred = r_k(y_test[i], y_pred[i] ,5, 5)
        avg_recall += skmetrics.recall_score(actual, pred_o, average='macro') # Recall@5
        avg_map += cal_ap(y_test[i], y_pred[i], 5)
    else:
        count +=1
        pass
    
print('AUC: ', avg_auc / y_test.shape[0])
print('MAP: ', avg_map / y_test.shape[0])
print('Recall@5: ', avg_recall / y_test.shape[0])
print('Skipped: {} rows, total: {} rows'.format(count, y_test.shape[0]))

# fpr, tpr, thresholds = skmetrics.roc_curve(y_test, pred_MNB_prob, pos_label=1) # Collect the recall and false positive rate from all 2000 predictions
# acc[0] += skmetrics.auc(fpr, tpr)
# # acc[1] += cal_ap(truth, predict, 5)